# Table of Contents

This notebook generates data used to contruct `binding peak` tables and `venn diagrams` for ChipDB. But before you run the notebook, make sure of the following: 

1. Concatnate files that contain multiple conditions for the same TF by using the following command on your terminal: `cat /path/prefix* >> new_file`.
2. Make sure to rename TF in the gff file to match the TF convention and not gene convension.

In [1]:
#basics:
import numpy as np     
import pandas as pd
import urllib                      
import gzip
import os
from os import path
from collections import defaultdict

# TCS Curated List (only need to run once for each TF)

In [2]:
# TF = 'KdpE'
# cond ='_KCl'

# peaks = [int(i[5:]) for i in pd.read_csv('./input_data/TCS_peaks/'+TF+'_annotated_curated.csv', index_col=0).peak_ID.unique()]
# peaks.sort()

# gff = pd.read_csv('./input_data/TCS_peaks/'+TF+'_COND_curated.gff',index_col=0, 
#                  delimiter='\t', header=None, 
#                  names = ['ref','condition', 'condition_name', 
#                           "binding_peak_start",'binding_peak_end', 
#                           'binding_peak_strength', 'direction', '.','ID'])

# gff_filtered = gff.iloc[peaks, :]

# gff_filtered.to_csv(in_gff+TF+cond+"_curated.gff", header=False, sep="\t")

# Binding Peak Tables

## Generate a data objects

### Prerequisite
Ensure the following paths reflect your local **input** and **output** directories

In [3]:
in_data = './input_data/'
in_gff = in_data + 'gff_data/'
out_data_venn = '../../data/e_coli/NC_000913_3/venn/'
out_data_table = '../../data/e_coli/NC_000913_3/table/'

In [4]:
database = defaultdict(list)

In [5]:
filelist = [file for file in os.listdir(in_gff)]
filelist

['CpxR_EtOH_curated.gff',
 'Ybdo_M9_curated.gff',
 'KdpE_KCl_curated.gff',
 'Yjdc_M9_curated.gff',
 'Ydip_M9_curated.gff',
 'Ycfq_M9_curated.gff',
 'Ydcn_M9_curated.gff',
 'Ycjw_M9_curated.gff',
 'Ycit_M9_curated.gff',
 'Ybcm_M9_curated.gff',
 'YdcI_ph5_curated.gff',
 'Ybaq_M9_curated.gff',
 'Ybef_M9_curated.gff',
 'Ybhd_M9_curated.gff',
 'RcsB_RcsD-KO_curated.gff',
 'AtoC_acetoacetate_curated.gff',
 'Ydhb_M9_curated.gff',
 'Yiep_M9_curated.gff',
 'Yheo_M9_curated.gff',
 'YdcI_ph7_curated.gff',
 'Yddm_M9_curated.gff',
 'Ydcr_M9_curated.gff',
 'ZraR_ZnCl2_curated.gff',
 'PhoB_M9P_curated.gff',
 'BtsR_pyruvate_curated.gff',
 'RcsB_EtOH_curated.gff',
 'Ycan_M9_curated.gff',
 'CusR_CuSO4_curated.gff',
 'BaeR_EtOH_curated.gff',
 'Ybao_M9_curated.gff',
 'YdcI_ph8_curated.gff',
 'Yahb_M9_curated.gff',
 'Yeam_M9_curated.gff']

In [6]:
filelist = [file for file in os.listdir(in_gff)]

for i in range(0,len(filelist)):
    df = pd.read_csv(path.join(in_gff,filelist[i]),index_col=0, 
                                 delimiter='\t', header=None, 
                                 names = ['ref','condition', 'condition_name', 
                                          "binding_peak_start",'binding_peak_end', 
                                          'binding_peak_strength', 'direction', '.','ID'])

    df = df.set_index(pd.Series(range(1,len(df)+1)))
    database[i] = df

In [7]:
database[13].head()

,condition,condition_name,binding_peak_start,binding_peak_end,binding_peak_strength,direction,.,ID
1,MACE,_filtered_0.95,480423,480436,1.14,+,.,GSB
2,MACE,_filtered_0.95,4068556,4068591,3.60,+,.,SFB
3,MACE,_filtered_0.95,4068576,4068609,3.57,+,.,SRB
4,MACE,_filtered_0.95,4274073,4274098,1.14,+,.,SRB


For the next block of code to work, make sure the files are named in the correct format: 
    `"<TF>_<CONDITION>_cutared.gff"`. The code will ensure TFs, genes, and conditions are in the correct conventions.

In [8]:
TF_list =[]
TF_condition = []
TF_gene_name = []
for i in range (0,len(filelist)): 
    split_file_name = filelist[i].split('_')
    
    first_upper_case = split_file_name[0].replace(split_file_name[0][0], split_file_name[0][0].upper())
    last_upper_case = first_upper_case.replace(first_upper_case[-1], first_upper_case[-1].upper())
    TF_list.append(last_upper_case)
    
    TF_condition.append(split_file_name[1])
    
    TF_gene_name.append(last_upper_case.lower())

In [9]:
TF_list

['CpxR',
 'YbdO',
 'KdpE',
 'YjdC',
 'YdiP',
 'YcfQ',
 'YdcN',
 'YcjW',
 'YciT',
 'YbcM',
 'YdcI',
 'YbaQ',
 'YbeF',
 'YbhD',
 'RcsB',
 'AtoC',
 'YdhB',
 'YieP',
 'YheO',
 'YdcI',
 'YddM',
 'YdcR',
 'ZraR',
 'PhoB',
 'BtsR',
 'RcsB',
 'YcaN',
 'CusR',
 'BaeR',
 'YbaO',
 'YdcI',
 'YahB',
 'YeaM']

## Load gene info + Biocyc TRN 

In [10]:
DF_EcoCyc_TF = pd.read_csv(path.join('./input_data','trn_EcoCyc.csv'),index_col=0)
DF_gene_info = pd.read_csv(path.join('./input_data','gene_info.csv'),index_col=0)
precise2_TRN = pd.read_csv(path.join('./input_data','trn_precise2.csv'),index_col=0)

In [11]:
# Define start codon position (to account for +/- strands)
DF_gene_info['start_codon_pos'] = [row.start if row.strand == '+' else row.stop for idx,row in DF_gene_info.iterrows()]
DF_gene_info.head()

,start,stop,strand,gene_name,length,operon,cog,start_codon_pos
b0001,189,255,+,thrL,66,thrLABC,No COG Annotation,189
b0002,336,2799,+,thrA,2463,thrLABC,No COG Annotation,336
b0003,2800,3733,+,thrB,933,thrLABC,Amino acid transport and metabolism,2800
b0004,3733,5020,+,thrC,1287,thrLABC,Amino acid transport and metabolism,3733
b0005,5233,5530,+,yaaX,297,yaaX,Function unknown,5233


## Generate Binding peaks 

This function assigns bindings peaks for each TF and fins the corresponsing gene targets using the `gene_info.csv` table. 

In [12]:
def annotate_peaks(TF_name, TF_condition, peak_df,margin,gene_info):
    
    res_df = peak_df.copy()
#     TF = peak_df.condition_name[1][:4]
    for i,row in res_df.iterrows():
        pos = row['binding_peak_start']
        # Identify genes within MARGIN nt of binding peak
        close_genes = gene_info[(gene_info.start_codon_pos > pos-margin) 
                                & (gene_info.start_codon_pos < pos+margin)]
        for strand,group in close_genes.groupby('strand'):
            #Remove genes that are completely transcribed before binding peak
            if strand == '+':
                group = group[group.stop > pos]
            else:
                group = group[group.start < pos]

            operon = group.operon.unique()
            # Ensure that we're only identifying one operon on either side of binding peak
            if len(operon) > 1:
                print (operon)

            # Get all genes in operon
            bnums = gene_info[gene_info.operon.isin(operon)].index
            
            ## Add gene information to dataframe
            if strand == '+':
                res_df.loc[i,'TU_p'] = ','.join(operon)
                res_df.loc[i,'genes_p'] = ','.join(bnums)
            else:
                res_df.loc[i,'TU_m'] = ','.join(operon)
                res_df.loc[i,'genes_m'] = ','.join(bnums)
    res_df['index'] = [TF_name +'-' + str(i) for i in range(1,peak_df.shape[0]+1)]
    res_df['condition'] = [ TF_name.lower() + " + " + TF_condition for i in peak_df.condition_name]
#     [peak_df.condition_name[2][:4]+' + '+peak_df.condition_name[1][5:8]]*peak_df.shape[0]
    cols = ['index','condition','binding_peak_start','binding_peak_end',
            'binding_peak_strength','TU_p','genes_p','TU_m','genes_m']
    return res_df.reindex(columns = cols)

## Validate Binding Peaks 

this function validates the accuracy of every gene target from the already indetified gene list for every binding site, identified from the previous function  

In [13]:
def validate_peak_info(df,gene_info):
    locusTag = defaultdict(list)
    geneName = defaultdict(list)
    for i,row in df.iterrows():
        BP = row['binding_peak_start']
        idx_name = i
        genes = [row['genes_p'] , row['genes_m']]
        if ((genes[0] == '') & (genes[1] == '')):
            locusTag[idx_name].append('')
            geneName[idx_name].append('')
        for gene in genes:
            if gene == '':
                continue 
            gene_list = gene.split(',')
            for g in gene_list: 
                name = DF_gene_info.loc[g].gene_name 
                strand = DF_gene_info.loc[g].strand 
                start = DF_gene_info.loc[g].start
                stop = DF_gene_info.loc[g].stop
                if ((start > BP) & (stop > BP) & (strand == '+')) | ((start < BP) & (stop < BP) & (strand == '-')):
                        locusTag[idx_name].append(g)
                        geneName[idx_name].append(name)
                elif ((start < BP) & (stop > BP)) | ((start > BP) & (stop < BP)):
                    locusTag[idx_name].append(g)
                    geneName[idx_name].append(name)
                    
    for k, v in locusTag.items():
        if ((len(v) == 1) & (v[0] == '')): 
            locusTag[k] = ''
            continue
        genes = ','.join(locusTag[k])
        locusTag[k] = genes

    for k, v in geneName.items():
        if ((len(v) == 1) & (v[0] == '')): 
            geneName[k] = ''
            continue
        genes = ','.join(geneName[k])
        geneName[k] = genes

    df_complete = df.loc[:,['index','condition','binding_peak_start',
                            'binding_peak_end','binding_peak_strength']]
    print(locusTag.values())
    df_complete['target_locus'] = locusTag.values()
    df_complete['target_genes'] = geneName.values()
    return df_complete

# Venn Diagrams

Make sure to update the list of `TF_names` as Ye adds more gff files into the dropbox

In [14]:
TRN_data = defaultdict(list)
TF_name = TF_list

for i in TF_name: 
    TRN_data[i] = [x for x in  list(precise2_TRN.gene_name[precise2_TRN.index == i])  if  str(x) != 'nan']

In [15]:
def Venn_data_gen(Peak_DF): 
    TF_name = Peak_DF['index'][1][:4]
    gene_list = [i for i in Peak_DF.target_genes if i != '']
    chip_data= ','.join(list(gene_list)).split(',')
    reg_data = TRN_data[TF_name]
    all_genes = [i  for i in chip_data if i not in reg_data] + reg_data
    
    
    TF = TF_name
    reg_genes=reg_data
    reg_only = []
    chip_genes=chip_data
    chip_only = []
    shared_genes=[]
    for i in all_genes: 
        if (i in reg_data) & (i not in chip_data):
            reg_only.append(i)
        elif (i in chip_data) & (i not in reg_data):
            chip_only.append(i)
        elif (i in chip_data) & (i in reg_data):
            shared_genes.append(i)
            
    values = [TF,
          len(reg_genes),
          len(reg_only),
          len(chip_genes),
          len(chip_only),
          len(shared_genes),
          len(all_genes)]
    
    index_name = ['TF',
              'reg_genes',
              'reg_only',
              'chip_genes',
              'chip_only',
              'shared_genes',
              'all_genes']
    
    genes = ['; '.join(precise2_TRN.source[precise2_TRN.index == TF_name].unique()),
         reg_genes,
         reg_only,
         chip_genes,
         chip_only,
         shared_genes,
         all_genes]
    
    same1 = defaultdict(list)
    for i in range(0,len(index_name)):
        same1[index_name[i]].append(values[i])

    finall = pd.DataFrame.from_dict(same1, orient='index', columns = ['value'])
    finall['list'] = genes
    for i, row in finall.iterrows(): 
        if row.value == 0:
            finall.list[i] = ''
    return finall

In [16]:
'; '.join(precise2_TRN.source[precise2_TRN.index == 'YdcI'].unique())

'Gao2021; Ecocyc'

In [17]:
for i in TF_list:
    print(i,len(TRN_data[i]))

CpxR 71
YbdO 4
KdpE 4
YjdC 0
YdiP 5
YcfQ 3
YdcN 61
YcjW 73
YciT 49
YbcM 12
YdcI 21
YbaQ 38
YbeF 3
YbhD 1
RcsB 26
AtoC 4
YdhB 28
YieP 6
YheO 8
YdcI 21
YddM 19
YdcR 19
ZraR 9
PhoB 61
BtsR 15
RcsB 26
YcaN 6
CusR 12
BaeR 9
YbaO 3
YdcI 21
YahB 3
YeaM 4


In [18]:
i = 16
print(TF_list[i])
peak_annot_DF = annotate_peaks(TF_list[i], TF_condition[i],database[i],500,DF_gene_info).fillna('')
final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
venn_files = Venn_data_gen(final_annot_DF)

YdhB
['frwCB' 'pflDC']
dict_values(['b0591,b0588,b0589,b0590', 'b0938,b0933,b0934,b0935,b0936,b0937', 'b1201,b1198,b1199,b1200', 'b1242,b1241', '', 'b1521', 'b1636,b1637,b1638', 'b1660', 'b1843,b1844', 'b2448,b2449', 'b2450', 'b2632,b2633', '', 'b3093,b3091,b3092', 'b3175', 'b3189,b3190', 'b3364,b3363', 'b4660_2', 'b3547', 'b3547', 'b3556', 'b3710', '', 'b3915', 'b3949,b3950,b3951,b3952', 'b4242,b4241', '', 'b4313'])


In [19]:
venn_files

,value,list
TF,YdhB,Gao2021
reg_genes,28,"[fimE, yjhV, treR, frwB, fieF, yihQ, mdtL, csp..."
reg_only,7,"[yjhV, yihQ, rpsJ, yggM, ydhB, purR, ydeO]"
chip_genes,49,"[entS, fepC, fepG, fepD, elfA, ssuB, ssuC, ssu..."
chip_only,27,"[fepC, fepG, fepD, elfA, ssuB, ssuC, ssuD, ssu..."
shared_genes,21,"[fimE, treR, frwB, fieF, mdtL, cspA, yhjX, ppi..."
all_genes,55,"[fepC, fepG, fepD, elfA, ssuB, ssuC, ssuD, ssu..."


# Binding Width Histograms

In [20]:
def binding_width_gen(TF, peak_df,out_dir): 
    widths = pd.DataFrame(peak_df.binding_peak_end - peak_df.binding_peak_start + 1, columns=["binding_width"])
    widths.to_csv(out_dir+TF+'_widths.csv')

# Peak Position Scatter

In [21]:
def peak_position_gen(TF, final_annot_DF,out_dir): 
    peak_scatter_df = pd.DataFrame(columns=final_annot_DF.columns.to_list()+['gene'] +['normalized_dist'])
    counter=0

    for i in range(len(final_annot_DF)):
        peak = final_annot_DF.loc[i+1,:]
        peak_center = np.mean([peak.binding_peak_start, peak.binding_peak_end])

        for gene in peak.target_genes.split(","):
            gene_info = DF_gene_info[DF_gene_info.gene_name == gene]
            try:
                if gene_info.strand[0] == "-":
                    dist = float((gene_info.stop - peak_center)/gene_info.length)
                else:
                    dist = float((peak_center - gene_info.start)/gene_info.length)
                peak_scatter_df.loc[counter] = peak.to_list()+[gene, dist]
                counter+=1
            except:
                continue

    peak_scatter_df['binding_peak_center'] = peak_scatter_df.binding_peak_start/2 + peak_scatter_df.binding_peak_end/2
    peak_scatter_df.to_csv(out_dir+TF+'_positions.csv')

# Generate the data

Run this block if you are ready to generate your data for ChIPdb.

In [22]:
for i in range (0, len(database)): 
    peak_annot_DF = annotate_peaks(TF_list[i], TF_condition[i],database[i],500,DF_gene_info).fillna('')
    final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
    venn_files = Venn_data_gen(final_annot_DF)
    final_annot_DF.to_json(out_data_table+TF_list[i]+'_binding_table.json',orient='records')
    venn_files.to_json(path.join(out_data_venn, TF_list[i]+'_venn.json'),orient='records')
    binding_width_gen(TF_list[i], database[i], '../../data/e_coli/NC_000913_3/binding_widths/')
    peak_position_gen(TF_list[i], final_annot_DF, '../../data/e_coli/NC_000913_3/positions/')

['yaiI' 'aroL-yaiA-aroM']
['raiA' 'pheLA']
['ygeG' 'ygeH']
dict_values(['b0113,b0114,b0115,b0116,b0112', 'b0118,b0117', 'b0161', 'b0278,b0279', 'b0278,b0279', 'b0278,b0279', 'b0281', 'b0281', 'b0287,b0283,b0284,b0285,b0286', '', 'b0387,b0388,b0389,b0390', 'b0390', 'b0460,b0461', 'b0602', 'b0610', 'b0629', 'b0632', 'b0656', 'b0733,b0734', 'b0733,b0734', 'b0925', 'b0953', 'b0970', 'b4490_1,b4490_2', 'b1056,b1057', 'b1177', 'b1205,b1203,b1204', 'b1205,b1203,b1204', 'b4522,b4523', 'b1552', 'b1552', 'b1586,b1585', 'b1593,b1594', 'b1712,b1713', 'b1785', 'b1814', 'b1891,b1892', 'b1891,b1892', 'b1973', 'b2175', 'b2216,b2217,b2215', 'b2232,b2231', 'b2232,b2231', 'b2276,b2277,b2278,b2279,b2280,b2281,b2282,b2283,b2284,b2285,b2286,b2287,b2288', 'b2448,b2449', 'b2471,b2472,b4547,b4606', 'b2504,b2503', 'b2504,b2503', 'b2574,b2570,b2571,b2572,b2573', 'b2597,b2598,b2599', '', '', 'b2650', 'b2754,b2755,b2756,b2757,b2758,b2759,b2760', 'b2794,b2793', 'b2851', 'b2851,b2852', 'b2910,b2906,b2907,b2908,b2909

dict_values(['b0721,b0722,b0723,b0724,b0726,b0727,b0728,b0729,b0720', 'b1521', 'b3172,b3164,b3165,b3166,b3167,b3168,b3169,b3170', 'b3175', 'b3556', ''])
dict_values(['b3556', 'b3829,b3828', 'b4322,b4323,b4321'])
['pdeB' 'ylaC']
['ydhR' 'ydhS']
dict_values(['b0034', '', 'b0217', 'b0281', 'b0457,b0458', 'b0527', 'b0601', '', 'b0771', 'b0792,b0793,b0794,b0795', 'b0826', '', 'b0923,b0924', '', 'b1197', 'b1201,b1198,b1199,b1200', 'b1221,b1222', 'b1418,b4493_2,b4493_1', 'b1434,b1433', 'b1521', '', '', 'b1661', 'b1667,b1668', 'b1722', 'b2099,b2100', '', '', 'b2448,b2449', '', '', 'b2513', 'b2570,b2571', '', 'b2890,b2891_2,b2891_1,b2892,b2893', 'b2970', 'b2988', '', 'b3062,b3063', '', '', 'b3228', 'b3228,b3229', 'b3228,b3229', 'b3255,b3256', '', 'b3540,b3541,b3542,b3543,b3544', 'b3556', 'b3623,b3624,b3625,b3626,b3627,b3628,b3629,b3630', '', '', 'b3990,b3991,b4407,b3992,b3993', 'b4084', 'b4135', 'b4322,b4323,b4321', ''])
['allA' 'allR']
['insH-8' 'yejO']
dict_values(['', 'b0066,b0067', 'b0114,b

['yegLK' 'ibsA']
['rpmH-rnpA' 'yidD']
['cpxPQ' 'fieF']
dict_values(['b0081,b0082,b0083,b0084,b0085,b0086,b0087,b0088,b0089,b0090,b0091,b0092,b0093,b0094,b0095,b0096', 'b0151,b0152,b0153', 'b0217', 'b0281', 'b0281', 'b0456,b0454', 'b0610', 'b0623', 'b0623', 'b0623', 'b0903,b0904', 'b0903,b0904', 'b0926,b0927', 'b0972,b0973,b0974,b0975,b0976,b0977,b0970', 'b1201,b1198,b1199,b1200', 'b1201,b1198,b1199,b1200', 'b1521', 'b1521', 'b1661', 'b1667,b1664', 'b1667,b1664', 'b1677', 'b1712,b1713,b1714,b1715,b1716,b1717', 'b1838', 'b1838', '', 'b1908', '', 'b1976', 'b1978', 'b1985', 'b2019,b2020,b2021,b2022,b2023,b2024,b2025,b2026,b4539,b2017', 'b2072,b2073,b4667', 'b2074,b2075,b2076,b2077,b2078,b2079,b4667', 'b2091,b2092_2,b2092_1,b2093', 'b2172', 'b2172', 'b2398,b2399,b2395', 'b2398,b2399,b2395', 'b2400', '', 'b2448,b2449', 'b2448,b2449', 'b2450', 'b2504,b2503', 'b2509,b2507,b2508', 'b2597', 'b2606,b2607,b2608,b2609', '', 'b2687', 'b2689,b2690', '', 'b2790,b2791,b2792', 'b2808', 'b2837,b2835,b283

['decR' 'mdlAB']
dict_values(['b0447,b0448,b0449', 'b4470,b3110'])
['holE' 'yobB-exoX']
dict_values(['b0019,b0020,b0018,b4412', 'b0076,b0071,b0072,b0073,b0074,b0075', 'b0281', 'b0382,b0381', 'b0720', 'b0770', 'b1418,b4493_2,b4493_1', 'b1634', 'b1823,b1824', 'b1842,b1843,b1844,b1839,b1840,b1841', 'b4496_1,b4496_2,b1962,b1963', 'b2210', '', 'b3267', 'b3603,b3604,b3605', 'b3646', 'b3710', 'b3800', 'b4077', 'b4145'])
dict_values(['b3065,b3066,b3067,b3064', 'b3172,b3164,b3165,b3166,b3167,b3168,b3169,b3170', 'b3211'])
dict_values(['b1201,b1198,b1199,b1200', 'b1791,b1790', 'b3800'])


In [23]:
database[i]

,condition,condition_name,binding_peak_start,binding_peak_end,binding_peak_strength,direction,.,ID
1,MACE,_filtered_0.95,1250883,1250921,3.738747,+,.,NaN
2,MACE,_filtered_0.95,1875914,1875959,4.890000,+,.,SFB
3,MACE,_filtered_0.95,3982780,3982824,5.060000,+,.,SFB
